<a href="https://colab.research.google.com/github/IIF0403/Thesis/blob/main/SimSiam_model_utkast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import drive

In [6]:
#FCN modellen fra prosjektoppgave
class FCN(nn.Module):
  def __init__(self):#, classes):
    super(FCN, self).__init__()
    self.conv1 = nn.Conv1d(1, 128, 9, padding=(9 // 2))
    self.bnorm1 = nn.BatchNorm1d(128)        
    self.conv2 = nn.Conv1d(128, 256, 5, padding=(5 // 2))
    self.bnorm2 = nn.BatchNorm1d(256)
    self.conv3 = nn.Conv1d(256, 128, 3, padding=(3 // 2))
    self.bnorm3 = nn.BatchNorm1d(128)        
    #self.classification_head = nn.Linear(128, classes)
    self.output_dim = 128

  def forward(self, x):
    b1_class = F.relu(self.bnorm1(self.conv1(x)))
    b2_class = F.relu(self.bnorm2(self.conv2(b1_class)))
    b3_class = F.relu(self.bnorm3(self.conv3(b2_class)))

    #features_class = torch.mean(b3_class, 2)   
    #out_class = self.classification_head(features_class)

    return b3_class

In [7]:
### SIMPLE SIAMESE REPRESENTATION LEANRING MODELLEN ###

## DISTANCE FUNCTION ##
#The distance to minimize  (Negative Cosine similarity)
def D(p, z):
  z = z.detach() #stop gradient
  p = p
  return - F.cosine_similarity(p, z, dim=1).mean()


## PROJECTION MLP ## ( in f(x))
#has 3 FC layers with BN, the output FC has no ReLU, hidden FC is 2048-d
class projection_MLP(nn.Module):
  def __init__(self, in_dim, hidden_dim=2048, out_dim=2048): 
    super().__init__()
    #Layer 1
    self.FC1 = nn.Sequential(
      nn.Linear(in_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True)  )
    #Layer 2
    self.FC2 = nn.Sequential(
      nn.Linear(hidden_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True)  )
    #Layer 3 (output)
    self.FC3 = nn.Sequential(
      nn.Linear(hidden_dim, out_dim),
      nn.BatchNorm1d(hidden_dim) )
  
  def forward(self, x):
    x1 = self.FC1(x)
    x2 = self.FC2(x1)
    x_out = self.FC3(X2)
    return x_out


## PREDICTION MLP ## h(z)
#has 2 FC layers, only BN and ReLU on hidden layer (explained why in SimSiam paper). 
# input: z = f(x) dim = 2048
# output: p = h(z) dim = 2048
class prediction_MLP(nn.Module):
  def __init__(self, in_dim=2048, hidden_dim=512, out_dim=2048): # bottleneck structure
    super().__init__()
    #Layer 1
    self.FC1 = nn.Sequential(
      nn.Linear(in_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True) )
    #Layer 2 (output)
    self.FC2 = nn.Linear(hidden_dim, out_dim)
  
  def forward(self, x):
    x1 = self.FC1(x)
    x_out = self.FC2(x1)
    return x_out


## SIMSIAM MODEL ##
class SimSiam(nn.Module):
  def __init__(self, backbone=FCN()): #take backbone as input
    super().__init__()
    self.backbone = backbone
    self.projector = projection_MLP(backbone.output_dim) 
    
    #Encoder; z = f(x)
    self.encoder = nn.Sequential( 
      self.backbone,
      self.projector )

    #Predictor; p = h(z)
    self.predictor = prediction_MLP()
  
  def forward(self, x1, x2):
    #x1, x2: augmentations of x
    f = self.encoder
    h = self.predictor

    z1, z2 = f(x1), f(x2)
    p1, p2 = h(z1), h(z2)

    #Symmetric loss
    Loss = D(p1,z2)/2 + D(p2,z1)/2 
    return Loss


In [8]:

#Importing Dataset class from other ipynb file
#!pip install import_ipynb
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks'

#!ls
import import_ipynb
from Dataset_class import *

Dataset_small = Datasets = ["ChlorineConcentration"]
Small_dataset = Timeseries_Dataset(Dataset_small, transform = transforms.Compose( [TwoSegments(), ToTensor()] ))
train_small, test_small = data_split(Small_dataset)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
Loading ' ChlorineConcentration ' from github


In [ ]:


epochs = 1
train_bs = 512
momentum = 0.9
weight_decay =0.0001
warmup_epochs = 0 #=10 for for batch sizes  > 1024
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_lr = 0.05
lr = (base_lr*train_bs)/256

model = SimSiam().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

train_loader = DataLoader(train_small, batch_size = train_bs, shuffle=True)
losses =[]

for e in range(epochs):
  for batch in enumerate(train_loader):
    x1_batch = batch[1]['aug1']
    x2_batch = batch[1]['aug2']

    model.zero_grad()
    loss = model.forward(x1_batch, x2_batch)
    #loss = model.forward(x1_batch.to(device,non_blocking=True), x2_batch.to(device,non_blocking=True)).mean()
    loss.backward()
    optimizer.step()








In [ ]:
model.forward(x1,x2).backward()
print("forward backwork check")



RuntimeError: ignored